# DES	simulation	assignment

### 0 Init

In [3]:
import simpy
import numpy as np
import random
import math
import statistics

In [151]:
def DES_queue(size, arrival_rate, service_rate, n, rate_prob=True, arrival_prob=True):
    """
    size = number of customers
    arrival_rate = rate of arrival of customers, lambda
    service_rate = rate of service of customers, mu
    n = number of servers
    """

    env = simpy.Environment()
    servers = simpy.Resource(env, capacity = n)

    inter_arrival_times = []
    service_times = []

    waiting_times = []


    def customer_processing(env, servers, arrival_rate, service_rate, rate_prob=True):
        ''' Process the customer'''
        arrival_time = env.now # time now

        with servers.request() as request:
            yield request  # wait untill the server is available
            wait_time = env.now - arrival_time
            waiting_times.append(wait_time)  # save the wait time
            # Service time
            if rate_prob:
                ds = random.expovariate(service_rate)
            else:
                print(rate_prob)
                ds = 1/service_rate
            service_times.append(ds)
            yield env.timeout(ds)

    def customer_generator(env, size , servers, arrival_rate, service_rate, rate_prob=True, arrival_prob=True):
        ''' Generates a costumer after a delay of the arrival time and processes the customer  '''
        for i in range(size):
            if arrival_prob:
                dt = random.expovariate(arrival_rate)
            dt = random.expovariate(1/arrival_rate)
            inter_arrival_times.append(dt)
            yield env.timeout(dt) # wait for the next customer
            costomer = customer_processing(env,servers,arrival_rate,service_rate, rate_prob=rate_prob)
            env.process(costomer)

    env.process(customer_generator(env, size, servers,arrival_rate,service_rate, arrival_prob=arrival_prob, rate_prob=rate_prob))

    env.run()
    
    mean_interarrival = statistics.mean(inter_arrival_times)
    mean_waiting = statistics.mean(waiting_times)
    mean_service = statistics.mean(service_times)

    return [mean_interarrival, mean_waiting, mean_service]


In [152]:
size = 100 # number of customers in simulation
system_load = 0.1 # p 
mu = 2 # service rate, capacity of the server
servers = [1,2,4] # number of servers
num_simulations = 10 # number of simulations

def lamda(n):
    return 1 / (n* system_load * mu)

results = {1: [], 2: [], 4: []}
results2 = {1: [], 2: [], 4: []}

for n in servers:
    for _ in range(num_simulations):
        result = DES_queue(size, lamda(n), mu, n)
        results[n].append(result[1])
        results2[n].append(result[2])
print(results2)

{1: [0.42299248640018766, 0.49554090505874687, 0.42716874205149213, 0.5291645246472477, 0.5847233535315632, 0.46662843453840985, 0.4069830963198311, 0.5019233096816234, 0.470076753213057, 0.47340569554244344], 2: [0.42427998605544937, 0.5409636681272414, 0.45147218198331357, 0.5766094769312241, 0.4602297614062601, 0.4228522755319277, 0.47970114890730264, 0.5427055140428056, 0.45207484256017083, 0.5594678693302931], 4: [0.4753660452928766, 0.4600696396220215, 0.4862268323670575, 0.5155708617611499, 0.579341258222902, 0.5390701858317016, 0.46761173335728506, 0.5481760300953887, 0.4504452360074183, 0.533830395108866]}


In [ ]:
import math

def calculate_expected_waiting_time(n, p, u):
    # Bereken rho
    if p >= 1:
        return "P DOES NOT WORK! DO NORMAL PLS :)"
    
    hfirst_term = ((n * p ) ** n) / math.factorial(n)
    
    hsecond_term = (1 - p) * sum(((n * p) ** i) / math.factorial(i) for i in range(n))
    hsecond_term += ((n * p) ** n) / math.factorial(n)
    hsecond_term = 1 / hsecond_term
    h = hfirst_term * hsecond_term
    second_term1 = 1 / (1 - p)
    second_term2 = 1 / (n * u)
    expected_waiting_time = h * second_term1 * second_term2

    return expected_waiting_time


def calculate_DES(number_of_rounds, arrival_rate, service_rate, number_of_servers, number_of_customers):
    results_waiting = []
    for _ in range(number_of_rounds):
        result = DES_queue(number_of_customers, arrival_rate, service_rate, number_of_servers, rate_prob=True, arrival_prob=True)
        results_waiting.append(result[1])
    # get avr over [_,x,_] 
    return np.mean(results_waiting), np.std(results_waiting)

lambd = 4
n = 2 # number of servers
u = 1 # service rate, capacity of the server
number_of_rounds = 100
number_of_customers = 1000
p = lambd / (n * u)
p= 0.7
lambd = 1/ (n * p * u)

result = calculate_expected_waiting_time(n,p,u)
result2 = calculate_DES(number_of_rounds, lambd, u, n, 10000)
print("Expected waiting time:", result)
print("Simulated waiting time:", result2)


Expected waiting time: 0.96078431372549
Simulated waiting time: (0.9715244413236764, 0.09306533903149226)
